# Install and Load Packages

In [1]:
install.packages("survey")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘minqa’, ‘numDeriv’, ‘mitools’




In [2]:
install.packages("readxl")
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(survey)
library(readxl)
library(dplyr)

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Load Data Susenas Maret 2021

In [4]:
listrik_jateng <- read_excel("Konsumsi_Listrik_Ruta_Jateng_Susenas_2021.xlsx")
# Retype variables
listrik_jateng$KodeProv <- as.factor(listrik_jateng$KodeProv)
listrik_jateng$KodeKabKot <- as.factor(listrik_jateng$KodeKabKot)
listrik_jateng$KodeKec <- as.factor(listrik_jateng$KodeKec)
listrik_jateng$KlasWil <- as.factor(listrik_jateng$KlasWil)
listrik_jateng$KodeSampel <- as.factor(listrik_jateng$KodeSampel)
listrik_jateng$idkec <- as.factor(listrik_jateng$idkec)
# Check missing data from each variable
sapply(listrik_jateng, function(x) sum(is.na(listrik_jateng)))
# Cek jumlah sampel tiap kecamatan
table(listrik_jateng$KodeProv, listrik_jateng$idkec)

KodeProv              KodeKabKot                 KodeKec 
                      0                       0                       0 
                  idkec                 KlasWil              KodeSampel 
                      0                       0                       0 
             NoUrutRuta     HasilPencacahanRuta               JumlahART 
                      0                       0                       0 
                   WERT                   WEIND                     PSU 
                      0                       0                       0 
                    SSU         KonsumsiListrik      PengeluaranListrik 
                      0                       0                       0 
KONSUMSI_LISTRIK_KAPITA 
                      0

    
     3301010 3301020 3301030 3301040 3301050 3301060 3301070 3301080 3301090
  33      70      99      79      79      50      49      30      30      40
    
     3301100 3301110 3301120 3301121 3301130 3301140 3301150 3301160 3301170
  33     100      40      40      20      49      40      40      10      19
    
     3301180 3301190 3301200 3301710 3301720 3301730 3302010 3302020 3302030
  33      50      30      30      30      30      20      70      50      40
    
     3302040 3302050 3302060 3302070 3302080 3302090 3302100 3302110 3302120
  33      39      20      50      40      19      30      30      30      30
    
     3302130 3302140 3302150 3302160 3302170 3302180 3302190 3302200 3302210
  33      30      70      50      50      70      29      40      20      49
    
     3302220 3302230 3302710 3302720 3302730 3302740 3303010 3303020 3303030
  33      40      30      30      20      30      30      80      70      49
    
     3303040 3303050 3303060 3303070 3303

# Estimasi Langsung

In [5]:
# Mengatur sampel PSU yang terisolasi dengan metode penyesuaian
options(survey.lonely.psu="adjust")
# Desain survei SUSENAS
susenas.design.jateng <- svydesign(ids = ~PSU+SSU, strata = ~KlasWil, data=listrik_jateng, weights = ~WEIND)
summary(susenas.design.jateng)
# Menghitung hasil estimasi
hasil <- svyby(formula= ~KONSUMSI_LISTRIK_KAPITA, ~idkec, design=susenas.design.jateng, deff=TRUE, svymean, vartype = c("se", "ci", "ci", "cv", "cvpct", "var"))
y_est_lgsg <- hasil$KONSUMSI_LISTRIK_KAPITA
var_est_lgsg <- hasil$var
se_est_lgsg <- hasil$se
ci_lower_est_lgsg <- hasil$ci_l
ci_upper_est_lgsg <- hasil$ci_u
deff_est_lgsg <- hasil$DEff.KONSUMSI_LISTRIK_KAPITA
rse_est_lgsg <- hasil$`cv%`
mse_est_lgsg <- hasil$var # MSE=VAR karena bias penduga langsung dianggap = 0

Stratified 2 - level Cluster Sampling design (with replacement)
With (3005, 29856) clusters.
svydesign(ids = ~PSU + SSU, strata = ~KlasWil, data = listrik_jateng, 
    weights = ~WEIND)
Probabilities:
     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
9.822e-05 6.343e-04 1.023e-03 2.153e-03 1.920e-03 2.123e-01 
First-level Stratum Sizes: 
               1     2
obs        15857 13999
design.PSU  1598  1407
actual.PSU  1598  1407
Data variables:
 [1] "KodeProv"                "KodeKabKot"             
 [3] "KodeKec"                 "idkec"                  
 [5] "KlasWil"                 "KodeSampel"             
 [7] "NoUrutRuta"              "HasilPencacahanRuta"    
 [9] "JumlahART"               "WERT"                   
[11] "WEIND"                   "PSU"                    
[13] "SSU"                     "KonsumsiListrik"        
[15] "PengeluaranListrik"      "KONSUMSI_LISTRIK_KAPITA"

Warning message in matrix(1:(ns * reps), ncol = reps, nrow = ns, byrow = TRUE):
“non-empty data for zero-extent matrix”


# Ekspor Hasil Estimasi Langsung

In [7]:
temp <- read_excel("data_estimasi_langsung.xlsx")
data_est_lgsg <- data.frame(temp$KodeKecamatan, temp$NamaKecamatan, y_est_lgsg, var_est_lgsg, se_est_lgsg, ci_lower_est_lgsg, ci_upper_est_lgsg, deff_est_lgsg, rse_est_lgsg, mse_est_lgsg)
names(data_est_lgsg) <- c("KodeKec", "NamaKec", "Konsumsi_Listrik_Kapita", "Var", "SE", "CI_Lower", "CI_Upper", "DEFF", "RSE", "MSE")
write.csv(data_est_lgsg, "Hasil_Estimasi_Langsung.csv")